<a href="https://colab.research.google.com/github/mildsupitcha25/MachineLearning_phishing_email/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import xgboost as xgb

## --- 1. การสร้างข้อมูลจำลอง (Simulated Stylometric Data) ---
# ในสถานการณ์จริง คุณจะต้องใช้ข้อมูล stylometric 60 ฟีเจอร์ที่สกัดจากอีเมล
# เช่น imperative_verbs_count, clause_density, first_person_pronoun_count
# โดยมีจำนวนตัวอย่างทั้งหมด 126 อีเมล (63 Phishing, 63 Legitimate)

n_samples = 126 # จำนวนตัวอย่างทั้งหมดในงานวิจัย (63 Phishing + 63 Legitimate) [cite: 143]
n_features = 60 # จำนวน Stylometric Features ที่ใช้ [cite: 304, 316]

# สร้าง DataFrame จำลอง (สมมติว่า 'X' คือ 60 ฟีเจอร์ที่ผ่านการสกัดแล้ว)
# สร้างข้อมูลที่มีความแตกต่างกันเล็กน้อยระหว่าง 2 Class เพื่อให้โมเดลสามารถเรียนรู้ได้
X = np.random.rand(n_samples, n_features) * 5
y = np.array([0] * (n_samples // 2) + [1] * (n_samples - n_samples // 2)) # 0: Legitimate, 1: Phishing

# ปรับให้ข้อมูล Phishing (y=1) มีค่าในฟีเจอร์สำคัญสูงขึ้นเล็กน้อย
# (เช่น ฟีเจอร์ 0 สมมติเป็น imperative_verbs_count)
X[y == 1, 0] = X[y == 1, 0] + 1.5
# (ฟีเจอร์ 1 สมมติเป็น clause_density)
X[y == 1, 1] = X[y == 1, 1] + 1.0


## --- 2. การเตรียมข้อมูล (Data Preprocessing) ---

# 2.1. การปรับมาตรฐาน (Feature Scaling)
# StandardScaler จะแปลงข้อมูลให้มีค่าเฉลี่ยเป็น 0 และส่วนเบี่ยงเบนมาตรฐานเป็น 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2.2. การแบ่งชุดข้อมูล (Train-Test Split)
# แบ่งเป็น Training 80% และ Testing 20%
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y)
# stratify=y ช่วยให้สัดส่วนของ Phishing/Legitimate ในชุด Train/Test ใกล้เคียงกัน


## --- 3. การฝึกโมเดล XGBoost (XGBoost Model Training) ---

# 3.1. กำหนดค่า Hyperparameters
# ใช้ค่าเริ่มต้นจาก Scikit-learn และตามที่งานวิจัยระบุ (eval_metric='logloss', random_state=42)
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# 3.2. ฝึกโมเดล
print(f"กำลังฝึก XGBoost บนข้อมูล {len(X_train)} ตัวอย่าง...")
xgb_model.fit(X_train, y_train)
print("ฝึกโมเดลเสร็จสิ้น")


## --- 4. การประเมินผล (Model Evaluation) ---

# 4.1. การทำนาย (Prediction)
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1] # สำหรับคำนวณ AUC

# 4.2. คำนวณเมตริกการประเมินผล
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() # ดึงค่าจาก Confusion Matrix

# 4.3. แสดงผลลัพธ์
print("\n" + "="*50)
print("✨ ผลลัพธ์การประเมินโมเดล XGBoost บนชุดทดสอบ 20% ✨")
print("="*50)
print(f"Accuracy: {accuracy:.4f} (งานวิจัยทำได้ 0.96) [cite: 576]")
print(f"Precision: {precision:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"Recall: {recall:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"F1-Score: {f1:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"AUC-Score: {auc_score:.4f} (งานวิจัยทำได้ 0.99) [cite: 601]")

print("\n--- Confusion Matrix ---")
print(f"True Negatives (TN): {tn}") # Legitimate ถูกทำนายถูก
print(f"False Positives (FP): {fp}") # Legitimate ถูกทำนายผิดเป็น Phishing
print(f"False Negatives (FN): {fn}") # Phishing ถูกทำนายผิดเป็น Legitimate
print(f"True Positives (TP): {tp}") # Phishing ถูกทำนายถูก [cite: 575]

กำลังฝึก XGBoost บนข้อมูล 100 ตัวอย่าง...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:09:19] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


ฝึกโมเดลเสร็จสิ้น

✨ ผลลัพธ์การประเมินโมเดล XGBoost บนชุดทดสอบ 20% ✨
Accuracy: 0.6923 (งานวิจัยทำได้ 0.96) [cite: 576]
Precision: 0.6667 (งานวิจัยทำได้ 0.96) [cite: 601]
Recall: 0.7692 (งานวิจัยทำได้ 0.96) [cite: 601]
F1-Score: 0.7143 (งานวิจัยทำได้ 0.96) [cite: 601]
AUC-Score: 0.7515 (งานวิจัยทำได้ 0.99) [cite: 601]

--- Confusion Matrix ---
True Negatives (TN): 8
False Positives (FP): 5
False Negatives (FN): 3
True Positives (TP): 10


#RandomForestClassifier

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest, chi2

# ✅ ใช้ลิงก์ raw file ของ GitHub
traindata = 'https://raw.githubusercontent.com/mildsupitcha25/MachineLearning_phishing_email/main/train.csv'

# --- 1. โหลดข้อมูล ---
df = pd.read_csv(traindata)

# ✅ ตรวจสอบคอลัมน์
print("คอลัมน์ที่มีอยู่:", df.columns.tolist())

# ✅ ถ้าชื่อคอลัมน์ไม่ตรง ให้แก้
if 'Subject' not in df.columns or 'Label' not in df.columns:
    raise ValueError("คอลัมน์ Subject หรือ Label ไม่พบในไฟล์ CSV")

# ✅ แปลง Label เป็นตัวเลข
df['Label_num'] = df['Label'].map({'Legitimate': 0, 'Phishing': 1})

X = df['Subject']
y = df['Label_num']

print(f"โหลดข้อมูลสำเร็จ. จำนวนอีเมล: {len(df)}")

# --- 2. แบ่งข้อมูล ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# --- 3. TF-IDF ---
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1,2), max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# --- 4. Feature Selection (เลือกฟีเจอร์ที่สำคัญ) ---
selector = SelectKBest(chi2, k=2000)
X_train_sel = selector.fit_transform(X_train_vec, y_train)
X_test_sel = selector.transform(X_test_vec)

# --- 5. โมเดล RandomForest ---
rf_model = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=42, class_weight='balanced')
rf_model.fit(X_train_sel, y_train)

# --- 6. ประเมินผล ---
y_pred = rf_model.predict(X_test_sel)
print("="*50)
print("✨ ผลลัพธ์การจำแนก Phishing ด้วย RandomForest ✨")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['Legitimate', 'Phishing']))

# --- 7. ทดสอบกับอีเมลใหม่ ---
new_email = ["Verify your account password now or it will be deleted.",
             "Just checking in on the report status, no rush."]
new_email_vec = vectorizer.transform(new_email)
new_email_sel = selector.transform(new_email_vec)
new_pred = rf_model.predict(new_email_sel)

print("\n--- การทำนายอีเมลใหม่ ---")
for i, pred in enumerate(new_pred):
    label = 'Phishing' if pred == 1 else 'Legitimate'
    print(f"อีเมล: '{new_email[i]}' -> ทำนายเป็น: {label}")


คอลัมน์ที่มีอยู่: ['Subject', 'Body', 'Label']
โหลดข้อมูลสำเร็จ. จำนวนอีเมล: 100


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=2000 is greater than n_features=396. All the features will be returned.
  warnings.warn(


✨ ผลลัพธ์การจำแนก Phishing ด้วย RandomForest ✨
Accuracy: 0.7667

Confusion Matrix:
 [[10  5]
 [ 2 13]]

Classification Report:
               precision    recall  f1-score   support

  Legitimate       0.83      0.67      0.74        15
    Phishing       0.72      0.87      0.79        15

    accuracy                           0.77        30
   macro avg       0.78      0.77      0.76        30
weighted avg       0.78      0.77      0.76        30


--- การทำนายอีเมลใหม่ ---
อีเมล: 'Verify your account password now or it will be deleted.' -> ทำนายเป็น: Phishing
อีเมล: 'Just checking in on the report status, no rush.' -> ทำนายเป็น: Phishing


#Naive Bayes

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ✅ ใช้ลิงก์ raw file
traindata = 'https://raw.githubusercontent.com/mildsupitcha25/MachineLearning_phishing_email/main/train.csv'

try:
    df = pd.read_csv(traindata)

    # ✅ ตรวจสอบคอลัมน์
    print("คอลัมน์ที่มีอยู่:", df.columns.tolist())

    # ✅ ถ้าชื่อคอลัมน์ไม่ตรง ให้แก้
    if 'Subject' not in df.columns or 'Label' not in df.columns:
        # ตัวอย่างการแก้ชื่อคอลัมน์
        # df.rename(columns={'Email_Content': 'Subject', 'Category': 'Label'}, inplace=True)
        raise ValueError("คอลัมน์ Subject หรือ Label ไม่พบในไฟล์ CSV")

    # ✅ แปลง Label เป็นตัวเลข
    if df['Label'].dtype == object:
        df['Label_num'] = df['Label'].map({'Legitimate': 0, 'Phishing': 1})
    else:
        df['Label_num'] = df['Label']

    X = df['Subject']
    y = df['Label_num']
    print(f"โหลดข้อมูลสำเร็จ. จำนวนอีเมล: {len(df)}")

except Exception as e:
    print(f"เกิดข้อผิดพลาดในการโหลดข้อมูล: {e}")

# ✅ แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# ✅ TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# ✅ โมเดล Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

# ✅ ประเมินผล
y_pred = nb_model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Legitimate', 'Phishing']))

คอลัมน์ที่มีอยู่: ['Subject', 'Body', 'Label']
โหลดข้อมูลสำเร็จ. จำนวนอีเมล: 100
Accuracy: 0.7666666666666667
Confusion Matrix:
 [[10  5]
 [ 2 13]]
Classification Report:
               precision    recall  f1-score   support

  Legitimate       0.83      0.67      0.74        15
    Phishing       0.72      0.87      0.79        15

    accuracy                           0.77        30
   macro avg       0.78      0.77      0.76        30
weighted avg       0.78      0.77      0.76        30

